In [1]:
import pandas as pd

In [2]:
df_sailor = pd.read_json("../data/official_brand_sites/sailor.json")
print(df_sailor.shape)
df_sailor.head(2)

(272, 6)


,brand,name,colors,color_names,description,details
0,sailor,Naginata Togi Ebonite Fountain Pen,[],[],[\n The once-a-year relea...,"{'Item Code': ['10-1813-378 (MF)', '10-1813-47..."
1,sailor,SHIKIORI Kusa Asobi Fountain Pen,"[#ffb6c1, #00bfff, #8b4513, #006400]","[Hanakanmuri, Sasabune, Tsurubami, Yukiusagi]",[\n This is a fountain pe...,"{'Item Code': ['［ Hanakanmuri ］', '11-0657-201..."


In [3]:
details = df_sailor['details'][0]
details

{'Item Code': ['10-1813-378 (MF)', '10-1813-478 (M)', '10-1813-678 (B)'],
 'Barcode': ['49-01680-17772-2 (MF)',
  '49-01680-17773-9 (M)',
  '49-01680-17774-6 (B)'],
 'Nib': ['21K Gold with Bicolor plating, Naginata Togi'],
 'Type': ['Converter & Cartridge type'],
 'Material': ['Ebonite'],
 'Size': ['φ20×152.1mm (including the clip)'],
 'Weight': ['36.2g (including the converter)'],
 'Package': ['Special gift box  (212×154×68mm)']}

In [4]:
keywords = ["nib", "type", "material", "size", "weight"]

In [5]:
test = df_sailor.copy()
            

In [6]:
def extract_keywords(details, keywords):
    extracted = {}
    for keyword in keywords:
        # Check each key in the 'details' dict for the keyword (case insensitive)
        for key, value in details.items():
            if keyword.lower() in key.lower():
                extracted[keyword] = value
                break  # We found the first match, no need to continue searching for this keyword
    return extracted

In [7]:
df_extracted = test['details'].apply(lambda details: extract_keywords(details, keywords))

In [8]:
df_expanded = pd.json_normalize(df_extracted)

In [9]:
df = pd.concat([test.drop(columns=['details']), df_expanded], axis=1)

In [10]:
def fix_weight(value):
    if isinstance(value, list):
        return float(value[0].split('g')[0])


In [11]:
df['weight'] = df['weight'].apply(fix_weight)

In [12]:
df.head(3)

,brand,name,colors,color_names,description,nib,type,material,size,weight
0,sailor,Naginata Togi Ebonite Fountain Pen,[],[],[\n The once-a-year relea...,"[21K Gold with Bicolor plating, Naginata Togi]",[Converter & Cartridge type],[Ebonite],[φ20×152.1mm (including the clip)],36.2
1,sailor,SHIKIORI Kusa Asobi Fountain Pen,"[#ffb6c1, #00bfff, #8b4513, #006400]","[Hanakanmuri, Sasabune, Tsurubami, Yukiusagi]",[\n This is a fountain pe...,"[Stainless Steel, Gold IP]",[Converter & Cartridge type],[PMMA Resin],[φ17×134mm（including the clip）],12.2
2,sailor,Teatime Around the World Fountain Pen Afternoo...,[],[],[\n Teatime Around the Wo...,[21K Gold with Rhodium plating],[Converter & Cartridge type],[PMMA Resin],[φ18 x 129mm (including the clip)],21.6


### most common weight

In [13]:
df['weight'].median()

np.float64(21.6)

In [14]:
df['colors'].value_counts()

colors
[]                                               91
[#000000]                                        30
[#ff0000]                                        15
[#8b0000]                                        11
[#00008b]                                        10
                                                 ..
[#ffb6c1, #32cd32, #ff8c00]                       1
[#ffffff, #006400]                                1
[#ffb6c1, #00008b, #ffff00, #006400, #ff8c00]     1
[#006400, #800080, #00bfff, #daa520]              1
[#ffffff, #ff0000, #191970, #006400]              1
Name: count, Length: 79, dtype: int64